In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
# filtered_survival_df.head()

(66, 16)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(66, 1023)


# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

In [11]:
def objective(h_params, X, y, r, verbose=True):
    if verbose:
        print(h_params)
    model = GradientBoostingRegressor(
        # We use lad since it most closely matches up with hyper-parameter objective
        loss="lad",
        # Use this for the same reason
        learning_rate=h_params[0],
        n_estimators=h_params[1],
        max_depth=h_params[2],
        max_features=h_params[3],
        min_samples_split=h_params[4],
        min_samples_leaf=h_params[5],
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="neg_mean_absolute_error"
    ))

In [12]:
space = [
    # We skip "ls" since we're optimizing for mean absolute error in the h_param opt.
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(6), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf")
]
n_initial = 10 * (len(space))
n_calls = 50 * (len(space))

In [13]:
callback_file = f"{unified_dsets[i]}_opt_gbr_h_params.tsv"
try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[0.07540275638507486, 533, 4, 'log2', 4, 1]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.6020
Function value obtained: 593.2486
Current minimum: 593.2486
Iteration No: 2 started. Evaluating function at random point.
[0.034738802219071975, 720, 5, 'log2', 4, 2]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.5388
Function value obtained: 532.8475
Current minimum: 532.8475
Iteration No: 3 started. Evaluating function at random point.
[0.08386822689969428, 343, 4, 'auto', 3, 3]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 4.2695
Function value obtained: 537.9290
Current minimum: 532.8475
Iteration No: 4 started. Evaluating function at random point.
[0.018853292027324983, 545, 3, 'log2', 3, 3]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.3516
Function value obtained: 528.6120
Current minimum: 528.6120
Iteration No: 5 started. Evaluatin

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 1.9019
Function value obtained: 519.9394
Current minimum: 506.3242
Iteration No: 75 started. Searching for the next optimal point.
[0.001, 604, 2, 'sqrt', 2, 1]
Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 2.3939
Function value obtained: 519.8503
Current minimum: 506.3242
Iteration No: 76 started. Searching for the next optimal point.
[0.025665608918387303, 257, 2, 'log2', 6, 2]
Iteration No: 76 ended. Search finished for the next optimal point.
Time taken: 1.4450
Function value obtained: 517.7709
Current minimum: 506.3242
Iteration No: 77 started. Searching for the next optimal point.
[0.02666491075087382, 260, 2, 'log2', 2, 2]
Iteration No: 77 ended. Search finished for the next optimal point.
Time taken: 1.7287
Function value obtained: 508.3725
Current minimum: 506.3242
Iteration No: 78 started. Searching for the next optimal point.
[0.022606842678408366, 266, 2, 'log2'

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 126 ended. Search finished for the next optimal point.
Time taken: 2.8595
Function value obtained: 521.6737
Current minimum: 500.1678
Iteration No: 127 started. Searching for the next optimal point.
[0.001, 100, 3, 'log2', 2, 1]
Iteration No: 127 ended. Search finished for the next optimal point.
Time taken: 2.6010
Function value obtained: 522.1682
Current minimum: 500.1678
Iteration No: 128 started. Searching for the next optimal point.
[0.001, 100, 2, 'log2', 6, 1]


...

Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 11.4587
Function value obtained: 514.6506
Current minimum: 500.1678
Iteration No: 299 started. Searching for the next optimal point.
[0.001, 1000, 5, 'log2', 2, 2]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 11.3236
Function value obtained: 514.2171
Current minimum: 500.1678
Iteration No: 300 started. Searching for the next optimal point.
[0.001, 1000, 5, 'log2', 2, 2]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 11.2848
Function value obtained: 514.1392
Current minimum: 500.1678
